## Import

In [1]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import ConcatDataset

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from ipywidgets import interact
from matplotlib import pyplot as plt

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import sklearn
from sklearn.ensemble import VotingClassifier

import timm
from torchvision.models import video
import torchvision

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

In [3]:
class CFG:
    model_name= "r3d_18"
    n_folds = 5
    n_classes = 13
    video_length=50
    img_size=128
    epochs=50
    lr=3e-4
    batch_size=16
    seed=41
    earlystop=10
    fold=5
        

## Fixed RandomSeed

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG.seed) # Seed 고정

## Data Load

In [5]:
df = pd.read_csv('./train_detail_classified.csv')
df.head()

,Unnamed: 0,sample_id,video_path,label,crush,ego,weather,timing
0,0,TRAIN_0000,./train/TRAIN_0000.mp4,7,1,0,0,0
1,1,TRAIN_0001,./train/TRAIN_0001.mp4,7,1,0,0,0
2,2,TRAIN_0002,./train/TRAIN_0002.mp4,0,0,-1,-1,-1
3,3,TRAIN_0003,./train/TRAIN_0003.mp4,0,0,-1,-1,-1
4,4,TRAIN_0004,./train/TRAIN_0004.mp4,1,1,1,0,0


## stratified K fold

In [6]:
#-1 라벨인 데이터 버리기
#df_crush=df[df['crush']!=1].reset_index(drop=True)
df_ego=df[df['ego']!=-1].reset_index(drop=True)
df_weather=df[df['weather']!=-1].reset_index(drop=True)
df_timing=df[df['timing']!=-1].reset_index(drop=True)
dfs=[(df, df['crush']),(df_ego, df_ego['ego']), (df_weather,df_weather['weather']), (df_timing,df_timing['timing'])]

In [7]:
skf = sklearn.model_selection.StratifiedKFold(n_splits=CFG.fold, shuffle=True, random_state=CFG.seed)
#라벨 분포에 맞춰서 fold, val 정보 넣기
for data in dfs:
    data[0]["fold"]=-1
    for k, (train_idx, val_idx) in enumerate(skf.split(*data)):
        data[0].loc[val_idx, 'fold']=k

In [40]:
df_weather[df_weather['fold']==0]['weather'].value_counts(dropna=False).sort_index()

0    144
1     25
2     14
Name: weather, dtype: int64

In [42]:
df_weather[df_weather['fold']==1]['weather'].value_counts(dropna=False).sort_index()

0    143
1     26
2     14
Name: weather, dtype: int64

## CustomDataset

In [8]:
#https://dacon.io/en/competitions/official/236064/codeshare/7572?page=1&dtype=recent#
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.label_list = label_list
        self.transforms = transforms
        self.img_path_list = img_path_list
        
    def __getitem__(self, index):        
        images = self.get_frames(self.img_path_list[index])
                        
        if self.transforms is not None:
            res = self.transforms(**images)
            #print(images)
            images = torch.zeros((len(images), 3, CFG.img_size, CFG.img_size))
            #print(images.shape)
            images[0, :, :, :] = torch.Tensor(res["image"])
            for i in range(1, len(images)):
                images[i, :, :, :] = res[f"image{i}"]
        
        images=images.permute(1,0,2,3)
        
        if self.label_list is not None:
            label = self.label_list[index]
        
            return images, label
        else:
            return images

    def __len__(self):
        return len(self.img_path_list) 
    
    def get_frames(self, path):
        cap = cv2.VideoCapture(path)
        frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        imgs = []        
        for fidx in range(frames):
            _, img = cap.read()            
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            imgs.append(img)
        
        ret = {f"image{i}":imgs[i] for i in range(1, len(imgs))}
        ret['image'] = imgs[0]

        return ret

In [9]:
rain_transforms = A.Compose([
    A.Resize(height=CFG.img_size, width=CFG.img_size),
    A.RandomRain(brightness_coefficient=0.9, drop_width=1, blur_value=3, p=0.3),
    A.Normalize(mean=0.0, std=1.0),
    ToTensorV2()
], p=1, additional_targets={f"image{i}":"image" for i in range(1, 50)})

snow_transforms = A.Compose([
    A.Resize(height=CFG.img_size, width=CFG.img_size),
    A.RandomSnow(brightness_coeff=2, snow_point_lower=0.3, snow_point_upper=0.5, p=0.3),
    A.Normalize(mean=0.0, std=1.0),
    ToTensorV2()
], p=1, additional_targets={f"image{i}":"image" for i in range(1, 50)})

other_transforms = A.Compose([
    #A.CenterCrop(480,854,p=1.0),
    A.Resize(height=CFG.img_size, width=CFG.img_size),
    #A.HorizontalFlip(0.5),
    A.Normalize(mean=0.0, std=1.0),
    ToTensorV2()
], p=1, additional_targets={f"image{i}":"image" for i in range(1, 50)})

test_transforms= A.Compose([
    #A.CenterCrop(480,854,p=1.0),
    A.Resize(height=CFG.img_size, width=CFG.img_size),
    #A.HorizontalFlip(0.5),
    A.Normalize(mean=0.0, std=1.0),
    ToTensorV2()
], p=1, additional_targets={f"image{i}":"image" for i in range(1, 50)})


In [10]:
#get data loader
def load_dataset(df,name,k,transforms= other_transforms, get_loader=True):
    
    train_df = df[df["fold"]!=k]
    val_df = df[df["fold"]==k]
    
    train_dataset = CustomDataset(df['video_path'].values, df[name].values,transforms=transforms)
    val_dataset = CustomDataset(df['video_path'].values, df[name].values,transforms=test_transforms)
    
    if get_loader==True:
        train_loader = DataLoader(train_dataset, batch_size = CFG.batch_size, shuffle=True, num_workers=2)
        val_loader = DataLoader(val_dataset, batch_size = CFG.batch_size, shuffle=False, num_workers=2)
        
        return train_dataset, val_dataset, train_loader, val_loader
    
    return train_dataset, val_dataset

In [11]:
#weather subset
def split_weather_subset(df=df_weather):
    
    df_rain=df[df['weather']==2]
    df_snow=df[df['weather']==1]
    df_normal=df[df['weather']==0]
    return df_normal, df_snow, df_rain


def weather_augment(df, k):

    
    tr=ConcatDataset([tr1,tr2,tr3])
    val=ConcatDataset([val1,val2,val3])
    
    train_loader = DataLoader(tr, batch_size = CFG.batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val, batch_size = CFG.batch_size, shuffle=False, num_workers=2)
    
    return tr, val, train_loader, val_loader

#a,b,_,_=weather_augment(df_weather, k=1)


In [12]:
df_weather["weather"].value_counts(dropna=False).sort_index()
#0번데이터 200개씩 랜덤하게 가져와 각각 증강하여 비, 눈 클래스에 맵핑하기 

0    716
1    129
2     70
Name: weather, dtype: int64

In [19]:
df_normal, df_snow, df_rain=split_weather_subset(df)

df_normal=df_normal.reset_index(drop=True)

In [32]:
sample_df=df_normal.sample(n=200, random_state=CFG.seed).reset_index(drop=True)

In [36]:
sample_df

,Unnamed: 0,sample_id,video_path,label,crush,ego,weather,timing,fold
0,978,TRAIN_0978,./train/TRAIN_0978.mp4,2,1,1,2,1,0
1,1178,TRAIN_1178,./train/TRAIN_1178.mp4,7,1,0,2,0,4
2,621,TRAIN_0621,./train/TRAIN_0621.mp4,7,1,0,2,0,2
3,2406,TRAIN_2406,./train/TRAIN_2406.mp4,1,1,1,2,0,1
4,1940,TRAIN_1940,./train/TRAIN_1940.mp4,2,1,1,2,1,1
...,...,...,...,...,...,...,...,...,...
195,2046,TRAIN_2046,./train/TRAIN_2046.mp4,1,1,1,2,0,0
196,2591,TRAIN_2591,./train/TRAIN_2591.mp4,7,1,0,2,0,4
197,48,TRAIN_0048,./train/TRAIN_0048.mp4,7,1,0,2,0,1
198,2213,TRAIN_2213,./train/TRAIN_2213.mp4,7,1,0,2,0,0


## Model Define
- 동일 모델로 순서대로 4개 사용

In [30]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=CFG.n_classes, fc_type='shallow', binary=True):
        super(BaseModel, self).__init__()
        self.fc_type=fc_type
        self.num_classes=num_classes
        
        #get backbone
        #self.backbone = r2plus1d_18(pretrained=True)
        self.backbone=getattr(torchvision.models.video,CFG.model_name)(pretrained=True)
        self.backbone.fc=self.get_fc()
        self.binary=binary
        
    def get_fc(self):
        if self.fc_type == 'deep':
            fc = nn.Sequential(nn.Linear(self.backbone.fc.in_features, self.backbone.fc.in_features//2),
                                        nn.BatchNorm1d(self.backbone.fc.in_features//2,  momentum=0.1),
                                        nn.ReLU(),
                                        nn.Linear(self.backbone.fc.in_features//2, self.num_classes)
                                        )
    
        elif self.fc_type == 'shallow':
            fc = nn.Linear(self.backbone.fc.in_features, self.num_classes)
        else:
            raise ValueError(f"Wrong fc-type input {self.fc_type}")
        return fc
    
    def forward(self, x):
        x = self.backbone(x)
            
        return x

### check model in-out

In [31]:
#from torchsummary import summary
#model=BaseModel(fc_type='deep')
#model=model.to(device)
#summary(model, (3,50,720,1280))

## Train

In [42]:
def train_one_epoch(model, criterion, optimizer, train_loader, val_loader, device):
    train_loss = []
    model.train()
    for videos, labels in tqdm(iter(train_loader)):
        videos = videos.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        output = model(videos)
        
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
        
    return train_loss # score 변화없으면 잘못된거니까 바꾸기 -> 업데이트를 안하는거겠지 옵티마이저 반환안해줘서

def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, trues = [], []
    
    with torch.no_grad():
        for videos, labels in tqdm(iter(val_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            #if model.module.binary==True: #because of DataParallel 
            #    labels=labels.float()
                
            logit = model(videos)
            
            #loss = criterion(logit, labels.reshape(-1,1))
            loss = criterion(logit, labels)
            val_loss.append(loss.item())
            
            preds += logit.argmax(1).detach().cpu().numpy().tolist() 

            trues += labels.detach().cpu().numpy().tolist()
        
        _val_loss = np.mean(val_loss)
    
    _val_score = f1_score(trues, preds, average='macro')
    return _val_loss, _val_score

In [43]:
def train(model, criterion, optimizer, train_loader, val_loader, scheduler, device, is_parallel=False):
    model.to(device)
    
    if torch.cuda.device_count() > 1:
        if is_parallel==True:
            print("Let's use", torch.cuda.device_count(), "GPUs!")
            model = nn.DataParallel(model)
    
    criterion=criterion().to(device)
    best_val_score = 0
    best_model = None
    cnt=0
    
    for epoch in range(1, CFG.epochs+1):
        train_loss=train_one_epoch(model, criterion, optimizer, train_loader, val_loader, device) #train 
        _val_loss, _val_score = validation(model, criterion, val_loader, device) #validation
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : [{_val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
            
            cnt=0
        else:
            print("early stopping count : {}".format(cnt+1))
            cnt+=1
        
        if best_val_score>=0.999:
            print("already on best score")
            break
        
        if cnt==CFG.earlystop:
            print("early stopping done")
            break
        
            
    return best_model

## Run!!

In [48]:
def run(model,df, name:str, transforms, device):
    for k in range(CFG.fold):
        if transforms=='weather':
            _,_,train_loader,val_loader=weather_augment(df, k)
        else:
            _,_,train_loader, val_loader = load_dataset(df,name, k , transforms= transforms)
        
        model.eval()
        optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG.lr)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=0.00001)
        
        criterion=nn.CrossEntropyLoss
        print("binary-classification")
        
        print("{} model run".format(name))
        print("{}th model run".format(k+1))
        print("_"*100)
        
        infer_model = train(model, criterion, optimizer, train_loader, val_loader, scheduler, device)
        torch.save(infer_model.state_dict(), './weights/r3d_per_label_{}_Transform_weather_{}fold_.pt'.format(name, k))
    
        del infer_model

In [49]:
model_crush = BaseModel(num_classes=2, fc_type='shallow')
model_ego = BaseModel(num_classes=2, fc_type='shallow')
model_weather = BaseModel(num_classes=3, fc_type='shallow', binary=False)
model_timing = BaseModel(num_classes=2, fc_type='shallow')

models=[model_crush, model_ego, model_weather, model_timing]
dfs=[(df, df['crush']),(df_ego, df_ego['ego']), (df_weather,df_weather['weather']), (df_timing,df_timing['timing'])]
names=["crush", "ego", "weather", "timing"]

run(models[2],dfs[2][0], names[2], transforms= 'weather', device=device)
#i=0
#for i in range(4):
#    for k in range(CFG.fold):
#        
#        #data load for each fold
#        _,_,train_loader, val_loader = load_dataset(dfs[i][0],names[i], k , transforms= other_transforms)
#    
#        model=models[i]
#        model.eval()
#        optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG.lr)
#        scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=0.00001)
#        
#        if i == 2: #weather
#            criterion=nn.CrossEntropyLoss
#            print("mutli-classification")
#        else:
#            criterion=nn.CrossEntropyLoss #여기 튜닝 고민 해봐야 할듯
#            print("binary-classification")
#        
#        print("{} model run".format(names[i]))
#        print("_"*100)
#        
#        infer_model = train(model, criterion, optimizer, train_loader, val_loader, scheduler, device)
#        torch.save(infer_model.state_dict(), './weights/r3d_per_label_{}_Transform_{}fold.pt'.format(names[i], k))
#    
#        del infer_model

binary-classification
weather model run
1th model run
____________________________________________________________________________________________________


  0%|          | 0/58 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.47444] Val Loss : [0.33778] Val F1 : [0.76285]


  0%|          | 0/58 [00:00<?, ?it/s]

...

  0%|          | 0/58 [00:00<?, ?it/s]

Epoch [4], Train Loss : [0.03010] Val Loss : [0.00776] Val F1 : [0.98659]
early stopping count : 1


  0%|          | 0/58 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

Epoch [5], Train Loss : [0.00824] Val Loss : [0.00256] Val F1 : [1.00000]
already on best score


## Inference

In [50]:
test = pd.read_csv('./test.csv')

In [51]:
test_dataset = CustomDataset(test['video_path'].values, None, transforms=test_transforms)
test_loader = DataLoader(test_dataset, batch_size = CFG.batch_size, shuffle=False, num_workers=0)

In [52]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    with torch.no_grad():
        for videos in tqdm(iter(test_loader)):
            videos = videos.to(device)
            
            logit = model(videos)

            preds += logit.argmax(1).detach().cpu().numpy().tolist()
    return preds

In [ ]:
i=0
from collections import OrderedDict

model_crush = BaseModel(num_classes=2, fc_type='shallow')
model_ego = BaseModel(num_classes=2, fc_type='shallow')
model_weather = BaseModel(num_classes=3, fc_type='shallow')
model_timing = BaseModel(num_classes=2, fc_type='shallow')
models=[model_crush, model_ego, model_weather, model_timing]
pred_tot={}
for i in range(4):
    preds_each=[]
    for k in range(CFG.fold):
        path='./weights/r3d_per_label_{}_Transform_{}fold.pt'.format(name[i], k)
        state_dict=torch.load(path)
        keys = state_dict.keys()
    
        values = state_dict.values()
        
        new_keys = []
        
        for key in keys:
            new_key = key[7:]    # remove the 'module.'
            new_keys.append(new_key)
        
        new_dict = OrderedDict(list(zip(new_keys, values)))
        
        model.load_state_dict(new_dict)
        
        preds = inference(model, test_loader, device) #순서대로 4종류씩 10번 담김
        preds_each.append(preds)
        
    pred_tot[names[i]]=(preds_each)
    del model

## Submission

In [ ]:
sample = pd.read_csv('./sample_submission.csv')

In [ ]:
from collections import Counter
from glob import glob

#voting
for k in range(CFG.fold):
    sample = pd.read_csv('./sample_submission.csv')
    for i in range(4):
        sample[names[i]] = pred_tot[names[i]][k]
    sample.to_csv(f'./ensemble/{names[i]}_fold{k}.csv', index=False)

for i in range(4):
    preds = []
    csvs = glob('./ensemble/{names[i]}_*.csv')
        
    for csv in csvs:
        f = pd.read_csv(csv)
        label = f[names[i]].tolist()
        preds.append(label)
    out = []
    cols = list(zip(*preds))
    for c in cols:
    most = Counter(c).most_common()[0][0]
    out.append(most)
    ss = pd.read_csv('./sample_submission.csv')
    ss[names[i]] = out
    ss.to_csv('vote_{}.csv'.format(names[i]), index=False) 

In [ ]:
#submit['crush'] = pred_tot[0]
#submit['ego'] = pred_tot[1]
#submit['weather'] = pred_tot[2]
#submit['timing'] = pred_tot[3]
#submit['label']=-1
#submit.head()

In [ ]:
submit["crush"].value_counts(dropna=False).sort_index()

In [ ]:
submit["ego"].value_counts(dropna=False).sort_index()

In [ ]:
submit["weather"].value_counts(dropna=False).sort_index()

In [ ]:
submit["timing"].value_counts(dropna=False).sort_index()

In [ ]:
#label ensemble
def label_ensemble(submit):
    for i, row in submit.iterrows():
        #print(i)
        if row['crush']==0: # 0
            submit['label'][i]=0 
        else:
            if row['ego']==1: # 1~6
                if row['weather']==0: #1,2
                    if row['timing']==0:
                        submit['label'][i]=1
                    else:
                        submit['label'][i]=2
                        
                elif row['weather']==1:# 3,4
                    if row['timing']==0:
                        submit['label'][i]=3
                    else:
                        submit['label'][i]=4
                else:
                    if row['timing']==0:# 5,6
                        submit['label'][i]=5
                    else:
                        submit['label'][i]=6
    
            else: # 7~12
                if row['weather']==0: #7,8
                    if row['timing']==0:
                        submit['label'][i]=7
                    else:
                        submit['label'][i]=8
                        
                elif row['weather']==1:# 9,10
                    if row['timing']==0:
                        submit['label'][i]=9
                    else:
                        submit['label'][i]=10
                else:
                    if row['timing']==0:# 11,12
                        submit['label'][i]=11
                    else:
                        submit['label'][i]=12
        if submit['label'][i]==-1:
            print(row['crush'], row['ego'], row['weather'], row['timing'])
    #라벨 추가했으니 필요없는 열 이제 삭제
    submit = submit.drop(['crush'],axis=1)
    submit = submit.drop(['ego'],axis=1)
    submit = submit.drop(['weather'],axis=1)
    submit = submit.drop(['timing'],axis=1)
    
    return submit

submit=label_ensemble(submit)

In [ ]:
submit.to_csv('./r3d_labelEnsemble_10fold_submit.csv', index=False)

In [ ]:
submit["label"].value_counts(dropna=False).sort_index()

In [ ]:
#2 7 8 없다